# AdaBrain-Bench Colab Quick Check (EEGNet + REVE)

This notebook pulls the repo, creates a tiny synthetic dataset, runs a quick EEGNet sanity check, and then runs REVE in dry-run mode on the same sample.

In [ ]:
!git clone https://github.com/iroblesrazzaq/AdaBrain-Bench.git
%cd AdaBrain-Bench
!pip install -q -r requirements.txt


In [ ]:
import sys
import subprocess

if sys.version_info >= (3, 12):
    subprocess.check_call([
        sys.executable, '-m', 'pip', 'install', '-q', '--upgrade', '--force-reinstall',
        'numpy>=2.0', 'scipy>=1.11'
    ])


In [ ]:
import json
import os
import pickle
from pathlib import Path

import numpy as np

root = Path('synthetic_data/SEED/multi_subject_json')
root.mkdir(parents=True, exist_ok=True)

C = 4
T = 200
rng = np.random.default_rng(0)

def make_samples(n, tag):
    samples = []
    for i in range(n):
        x = rng.standard_normal((C, T)).astype(np.float32)
        y = float(i % 2)
        path = root / f'{tag}_sample_{i}.pkl'
        with open(path, 'wb') as f:
            pickle.dump({'X': x, 'Y': y}, f)
        samples.append({'file': str(path), 'label': int(y), 'subject_id': 0})
    return samples

train = make_samples(12, 'train')
val = make_samples(4, 'val')
test = make_samples(4, 'test')

all_samples = train + val + test
stack = np.stack([pickle.load(open(s['file'], 'rb'))['X'] for s in all_samples], axis=0)
mean = stack.mean(axis=(0, 2)).tolist()
std = stack.std(axis=(0, 2)).tolist()
minv = stack.min(axis=(0, 2)).tolist()
maxv = stack.max(axis=(0, 2)).tolist()

dataset_info = {
    'sampling_rate': 200,
    'ch_names': [f'Ch{i + 1}' for i in range(C)],
    'mean': mean,
    'std': std,
    'min': minv,
    'max': maxv,
}

def write_split(name, data):
    payload = {'dataset_info': dataset_info, 'subject_data': data}
    with open(root / f'{name}.json', 'w') as f:
        json.dump(payload, f)

write_split('train', train)
write_split('val', val)
write_split('test', test)

config_path = Path('dataset_config/Classification.json')
if config_path.exists():
    with open(config_path, 'r') as f:
        config = json.load(f)
else:
    config = {}

config['SEED'] = {
    'root': {'multi': str(root), 'cross': str(root), 'fewshot': str(root)},
    'num_classes': 2,
    'num_t': 1,
}

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print('Synthetic dataset ready at', root)


In [ ]:
!python run_finetuning.py --model_name EEGNet --dataset SEED --task_mod Classification --subject_mod cross --finetune_mod full --norm_method z_score --batch_size 4 --epochs 1 --lr 1e-3 --sampling_rate 200 --seed 0 --num_workers 0 --device cpu


## REVE dry-run
If the model is gated for your account, authenticate once with `huggingface-cli login`.

In [ ]:
# Optional login if access is gated
# !huggingface-cli login


In [ ]:
!python run_finetuning.py --model_name REVE --dataset SEED --task_mod Classification --subject_mod cross --finetune_mod full --norm_method z_score --batch_size 4 --epochs 1 --lr 1e-3 --sampling_rate 200 --seed 0 --num_workers 0 --device cpu --dry_run


In [ ]:
!ls -la finetuning_results/Classification || true
